In [134]:
import csv
import pandas as pd
import numpy as np


In [346]:
df=pd.read_csv("BWFakeData.csv")
df.head(20)

,category,annualSpent,ads,aur,upt,numberDept,age,income,prevCategory
0,0,1765.654128,138.043031,51.375412,2.184242,8.059339,48.603100,5929.269951,0
1,0,1779.235457,141.101633,53.855448,1.792615,9.327111,51.877461,42232.922614,1
2,0,1791.259340,138.903619,52.842852,2.413194,13.692749,40.653785,7334.415152,0
3,0,1894.025873,143.090405,50.879810,3.718243,5.718444,44.801199,1476.881276,0
4,0,1754.563059,136.757503,49.715023,3.384003,13.361783,46.135527,2100.226851,0
5,0,1818.565878,140.828628,50.040162,2.404207,4.984910,45.814643,18724.175651,1
6,0,1775.553064,138.314849,51.097919,3.260138,8.313227,42.890914,2892.572459,1
7,0,1744.784080,143.378833,56.585813,1.843012,15.980275,42.170368,23304.702718,2
8,0,1791.998302,136.860752,51.000622,2.384818,9.841851,48.497411,1423.804713,1
9,0,1817.856526,133.350156,41.431236,1.579885,0.644297,46.682616,105103.659793,1


In [406]:
migrationsStacked = df.groupby(['prevCategory','category']).agg({'annualSpent' : 'count'})
migrations = migrationsStacked.unstack(level=-1)
migrations.columns= migrations.columns.droplevel()
migrations = migrations.fillna(0)
migrations

category,0,1,2,3,4
prevCategory,,,,,
0,24.0,52.0,53.0,6.0,3.0
1,21.0,86.0,136.0,32.0,22.0
2,4.0,29.0,147.0,83.0,47.0
3,0.0,3.0,57.0,93.0,42.0
4,1.0,0.0,7.0,29.0,26.0
5,0.0,0.0,0.0,7.0,10.0


In [407]:
migrationsToProspect = migrations.sum(axis=1)*0.06
migrations= pd.concat([migrations,migrationsToProspect ],axis=1)
migrations.columns = [0, 1, 2, 3, 4, 5]
migrations = migrations.apply(lambda x: 100*x/float(x.sum()),axis=1).round(2)
print migrations
migrations.T.to_json()

                  0      1      2      3      4     5
prevCategory                                         
0             16.41  35.55  36.23   4.10   2.05  5.66
1              6.67  27.32  43.20  10.16   6.99  5.66
2              1.22   8.83  44.74  25.26  14.30  5.66
3              0.00   1.45  27.58  44.99  20.32  5.66
4              1.50   0.00  10.48  43.43  38.93  5.66
5              0.00   0.00   0.00  38.85  55.49  5.66


'{"0":{"0":16.41,"1":35.55,"2":36.23,"3":4.1,"4":2.05,"5":5.66},"1":{"0":6.67,"1":27.32,"2":43.2,"3":10.16,"4":6.99,"5":5.66},"2":{"0":1.22,"1":8.83,"2":44.74,"3":25.26,"4":14.3,"5":5.66},"3":{"0":0.0,"1":1.45,"2":27.58,"3":44.99,"4":20.32,"5":5.66},"4":{"0":1.5,"1":0.0,"2":10.48,"3":43.43,"4":38.93,"5":5.66},"5":{"0":0.0,"1":0.0,"2":0.0,"3":38.85,"4":55.49,"5":5.66}}'

In [427]:

catAnnualSpent = df.groupby('category').sum()['annualSpent']
catAnnualSpent[5]=0
annualSpentAvg = catAnnualSpent/migrations.sum(axis=0)
annualSpentAvg[5] = 0
annualSpentAvg


category
0    3480.781628
1    1392.041119
2     590.798680
3     169.120763
4      34.311947
5       0.000000
dtype: float64

In [412]:
prospectPool = 10000
catCount = df.groupby('category').count()['annualSpent']
catCount[5]= prospectPool - catCount.sum()
catCount

category
0      50
1     170
2     400
3     250
4     150
5    8980
Name: annualSpent, dtype: int64

In [418]:
50*16.41+170*6.67+400*1.22+150*1.5
93.026*16.41+123.937*6.67+1798.5253*1.22+3151.768*1.5

9275.069316000001

In [422]:
catCount2 = (catCount.values).dot(migrations.values/100)
catCount3 = (catCount2).dot(migrations.values/100)

print catCount2
print catCount3


[   26.674   103.164   355.185  3786.712  5162.305   566.   ]
[   93.0260742   123.9371713  1798.5253408  4266.8166177  3151.7680505
   566.002264 ]


In [425]:
# Matrix multiplication
catAnnualSpentEvolution=[]
catAnnualSpentEvolution.append(annualSpentAvg*catCount)
prevCount = (catCount.values).dot(migrations.values/100)
catAnnualSpentEvolution.append(annualSpentAvg*prevCount)

for i in range(10):
    prevCount = prevCount.dot(migrations.values/100)
    catAnnualSpentEvolution.append(annualSpentAvg*prevCount)
pd.DataFrame(catAnnualSpentEvolution).sum(axis=1)

0     6.944325e+05
1     1.263838e+06
2     2.388646e+06
3     2.905288e+06
4     3.164553e+06
5     3.287864e+06
6     3.344574e+06
7     3.370260e+06
8     3.381838e+06
9     3.387068e+06
10    3.389456e+06
11    3.390573e+06
dtype: float64

In [396]:
catAnnualSpentEvoSum = pd.DataFrame(catAnnualSpentEvolution).sum(axis=1)
catAnnualSpentEvoSum2 = catAnnualSpentEvoSum
newdf = pd.DataFrame({"Default":catAnnualSpentEvoSum,"Alt1":catAnnualSpentEvoSum2,"Year": [i for i in range(12)]})
newdf.to_json(orient="records")

'[{"Alt1":320422.6893391034,"Default":320422.6893391034,"Year":0},{"Alt1":320403.526141719,"Default":320403.526141719,"Year":1},{"Alt1":320386.1641411135,"Default":320386.1641411135,"Year":2},{"Alt1":320370.1101604414,"Default":320370.1101604414,"Year":3},{"Alt1":320354.9480427362,"Default":320354.9480427362,"Year":4},{"Alt1":320340.2992440584,"Default":320340.2992440584,"Year":5},{"Alt1":320325.9325620057,"Default":320325.9325620057,"Year":6},{"Alt1":320311.7190047592,"Default":320311.7190047592,"Year":7},{"Alt1":320297.5883428791,"Default":320297.5883428791,"Year":8},{"Alt1":320283.5026250606,"Default":320283.5026250606,"Year":9},{"Alt1":320269.4413933372,"Default":320269.4413933372,"Year":10},{"Alt1":320255.3936289658,"Default":320255.3936289658,"Year":11}]'

In [335]:
import numpy as np
np.random.rand(11)*2000

array([  969.3421464 ,  1381.65898853,  1890.71798916,   849.05514994,
        1189.8874992 ,  1006.41564519,  1673.14265013,   657.49515612,
        1928.27026313,  1932.17029624,    50.66001185])